In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 웹 브라우저와 연동을 위해
from selenium import webdriver
# Chrome 객체의 인자로 넣기 위해
from selenium.webdriver.chrome.service import Service
# 사용 중인 Chrome version과의 싱크를 맞추기 위해
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.by import By
import time

In [2]:
cinemaID_dict = {}

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://www.lottecinema.co.kr/NLCHS/Ticketing/Schedule")
    driver.implicitly_wait(10)
    
    cinemaID = []
    for i in range(1, 25): # 1~24 서울 영화관 고유 아이디 추출
        element = driver.find_element(By.XPATH, '//*[@id="nav"]/ul/li[3]/div/ul/li[2]/div/ul/li[{}]/a'.format(i))
        
        href = element.get_attribute("href")  # href 속성 값을 가져옵니다.
        cinemaID.append(href[-4:])  # href 값을 출력합니다.
        
    cinemaID_dict["서울"] = cinemaID
    
    cinemaID = []
    for i in range(1, 48): # 1~48 경기/인천 영화관 고유 아이디 추출
        element = driver.find_element(By.XPATH, '//*[@id="nav"]/ul/li[3]/div/ul/li[3]/div/ul/li[{}]/a'.format(i))
        
        href = element.get_attribute("href")  # href 속성 값을 가져옵니다.
        cinemaID.append(href[-4:])  # href 값을 출력합니다.
        
        cinemaID_dict["경기/인천"] = cinemaID
        
print(cinemaID_dict)

{'서울': ['1013', '9094', '9010', '1004', '1009', '1003', '1023', '1017', '9056', '1012', '9099', '9104', '1024', '1015', '1007', '1001', '1002', '1014', '1016', '1021', '9083', '1008', '1010', '1005'], '경기/인천': ['3030', '3027', '3025', '3020', '3026', '3048', '9095', '3021', '3046', '3017', '3011', '9054', '3003', '3050', '3008', '3045', '3031', '3043', '3041', '3012', '3029', '3024', '3044', '9088', '3049', '3004', '3028', '9106', '3007', '3032', '9077', '9079', '3039', '3040', '3037', '3033', '9100', '3035', '3038', '9087', '3010', '3034', '3047', '3018', '9075', '9111', '3036']}


In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import date

today = date.today()
today_date = today.strftime("%Y-%m-%d")

url = "https://www.lottecinema.co.kr/LCWS/Ticketing/TicketingData.aspx"
citys = {"서울" : "0001", "경기/인천" : "0002"}
all_movie = []

for city in citys:
    cinemaID_list = cinemaID_dict[city] # ['1013', '9094', '9010', '1004', ...
    city_id = citys[city] # 0001
    for cinema_id in cinemaID_list:
        time.sleep(1) # 요청 전에 딜레이를 줌
        dic = {"MethodName":"GetPlaySequence",
           "channelType":"HO",
           "osType":"",
           "osVersion":"",
           "playDate":today_date,   # 오늘 날짜로 변경 필요!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
           "cinemaID":"1|{}|{}".format(city_id, cinema_id),
           "representationMovieCode":""
        }
        parameters = {"paramList": str(dic)}    
        response = requests.post(url, data = parameters).json()
        movies_response = response['PlaySeqs']['Items']


        for move_res in movies_response:
            # 상영 시작 시간과 종료 시간을 시간 형식으로 변환
            start_hour, start_minute = map(int, move_res['StartTime'].split(':'))
            end_hour, end_minute = map(int, move_res['EndTime'].split(':'))

            # 러닝타임 계산
            running_time = (end_hour - start_hour) * 60 + (end_minute - start_minute)
            move_data = {"theater_type":"롯데시네마", "theater_name":move_res['CinemaNameKR'], "location":city, "movie_title":move_res['MovieNameKR'], "start_time":move_res['StartTime'], "running_time":running_time}

            all_movie.append(move_data)
            print(move_data)
            
        print("------------------------------------------------------------------------------------------------------")


{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '12:15', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '15:10', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '18:05', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '21:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '17:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title'

{'theater_type': '롯데시네마', 'theater_name': '강동', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '강동', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '12:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '강동', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '15:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '강동', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '18:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '강동', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '21:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '강동', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '강동', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volum

{'theater_type': '롯데시네마', 'theater_name': '김포공항', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '김포공항', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '13:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '김포공항', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '16:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '김포공항', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '19:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '김포공항', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '22:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '김포공항', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:15', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '김포공항', 'location': '서울', 'movie_title': '가디언즈

{'theater_type': '롯데시네마', 'theater_name': '노원', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '20:50', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '노원', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '23:45', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '노원', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '15:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '노원', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '17:55', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '노원', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '노원', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:45', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '노원', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volum

{'theater_type': '롯데시네마', 'theater_name': '도곡', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '16:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '도곡', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '16:40', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '도곡', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '19:40', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '도곡', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '도곡', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '13:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '도곡', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '20:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '도곡', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volum

...

{'theater_type': '롯데시네마', 'theater_name': '인천아시아드', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '11:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '인천아시아드', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '14:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '인천아시아드', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '17:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '인천아시아드', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '20:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '인천아시아드', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '23:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '인천아시아드', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '인천아시아드', 'locat

{'theater_type': '롯데시네마', 'theater_name': '주엽', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '주엽', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '13:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '주엽', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '16:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '주엽', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '19:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '주엽', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '11:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '주엽', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '14:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '주엽', 'location': '경기/인천', 'movie_title'

{'theater_type': '롯데시네마', 'theater_name': '판교(창조경제밸리)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '14:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '판교(창조경제밸리)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '17:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '판교(창조경제밸리)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '20:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '판교(창조경제밸리)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '판교(창조경제밸리)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '13:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '판교(창조경제밸리)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '16:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater

{'theater_type': '롯데시네마', 'theater_name': '평택비전(뉴코아)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:50', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '평택비전(뉴코아)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '13:40', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '평택비전(뉴코아)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '16:30', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '평택비전(뉴코아)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '19:20', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '평택비전(뉴코아)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '22:10', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '평택비전(뉴코아)', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:50', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name'

{'theater_type': '롯데시네마', 'theater_name': '향남', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '향남', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '12:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '향남', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '15:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '향남', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '18:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '향남', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '21:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '향남', 'location': '경기/인천', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '10:00', 'running_time': 160}
{'theater_type': '롯데시네마', 'theater_name': '향남', 'location': '경기/인천', 'movie_title'

In [4]:
print(len(all_movie))

2524


In [5]:
print(all_movie)

[{'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '09:20', 'running_time': 160}, {'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '12:15', 'running_time': 160}, {'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '15:10', 'running_time': 160}, {'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '18:05', 'running_time': 160}, {'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '21:00', 'running_time': 160}, {'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie_title': '가디언즈 오브 갤럭시: Volume 3', 'start_time': '17:20', 'running_time': 160}, {'theater_type': '롯데시네마', 'theater_name': '가산디지털', 'location': '서울', 'movie